In [1]:
name='p2'
repo='NeuralNook'

In [2]:
!pip install -q duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.7 MB/s eta 0:00:00


In [3]:
from fastcore.all import * 
from duckduckgo_search import ddg_images

In [4]:
def search_images(term,max_images=30):
    print(f"Searching for {term}")
    return L(ddg_images(term,max_results=max_images)).itemgot('image')


In [5]:
urls=search_images('birds photos',max_images=1)
urls[0]

Searching for birds photos


'http://hdqwalls.com/download/1/colorful-parrot-bird.jpg'

In [ ]:
from fastdownload import download_url 

dest='bird.jpg'
download_url(url=urls[0],dest=dest,show_progress=False) 

from fastai.vision.all import * 

im=Image.open('bird.jpg')
im.to_thumb(256,256)

In [ ]:
download_url(search_images('forest photos',max_images=1)[0], dest='forest.jpg',show_progress=False)

im=Image.open('forest.jpg')
im.to_thumb(256,256)

In [ ]:
searches='bird','forest'

path=Path('bird_or_not')
from time import sleep 

for o in searches:
    dest=(path/o)
    dest.mkdir(exist_ok=True,parents=True)
    download_images(dest,urls=search_images(f'{o} photo'))
    sleep(10)
    download_images(dest,urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest,urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o , max_size=400, dest=path/o)
    


In [ ]:
failed=verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls=DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2),
    get_y=parent_label,
    item_tfms=[Resize(196,method='squish')]
).dataloaders(path,bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn=vision_learner(dls,resnet18,metrics=error_rate)
learn.fine_tune(3)

In [ ]:
is_bird,_,probs=learn.predict(im)
print(f"Is it a bird: '{is_bird}'")
print(f'prob that it is bird: {probs:.4f}')

In [ ]:
! cp /content/{name}.ipynb "$(date +"%Y_%m_%d_%I_%M_%p").ipynb"